# **0.2 Construcción del Dataset**

In [37]:
# Importamos librerías 
import pandas as pd
import os

In [38]:
# Cargar datos 
data_dir = os.path.join("..","data","processed","raw_cleaned")

ppi_c = pd.read_csv(os.path.join(data_dir, "ppi_clean.csv"))
target_c = pd.read_csv(os.path.join(data_dir, "target_clean.csv"))
topo_c = pd.read_csv(os.path.join(data_dir, "topo_clean.csv"))
go_c = pd.read_csv(os.path.join(data_dir, "go_enrichment_clean.csv"))


## Construir de edge.txt


In [39]:
output_dir = os.path.join("..", "data", "Input")
ppi_c.to_csv(os.path.join(output_dir, "Edge.csv"), sep='\t', index=False, header=False)

## Construir matriz P x Go_term

Conjunto de proteínas en estudio P.

Procesar anotaciones GO (Construye G)

In [40]:
# Separar proteínas asociadas a cada término GO
go_expanded = go_c.copy()
go_expanded['Overlapping genes'] = go_expanded['Overlapping genes'].astype(str).str.split(',')
go_expanded = go_expanded.explode('Overlapping genes')
go_expanded['Overlapping genes'] = go_expanded['Overlapping genes'].str.strip()
go_expanded = go_expanded.rename(columns={'Overlapping genes': 'Protein'})

Matriz funcional X_GO. donde filas = proteinas y columnas = términos GO.

In [41]:
# Matriz funcional: 1 si proteína tiene GO, 0 si no
protein_go_matrix = pd.crosstab(go_expanded['Protein'], go_expanded['GO'])

Tratamiento para complejos proteicos. Consutruye participantes (p_i) de cada complejo proteico.

In [42]:
# Filtrar complejos proteicos y extraer proteínas participantes
complex_df = target_c[target_c["Target_type"] == "PROTEIN COMPLEX"].copy()
complex_df = complex_df[complex_df["Complex_participants"].notnull()]
complex_df["Complex_participants"] = complex_df["Complex_participants"].astype(str).str.split(",")
complex_df = complex_df.explode("Complex_participants")
complex_df["Complex_participants"] = complex_df["Complex_participants"].str.strip()


Asignar GO a complejos proteicos por herencia. Aqui se hace X_GO[p_i, p_j]

In [43]:
# Mapeo: participante → complejo
complex_map = complex_df.rename(columns={"Complex_participants": "Protein", "Node_id": "Complex_id"})
complex_map = complex_map[["Protein", "Complex_id"]]

# Herencia de GO: participante → complejo
go_with_complex = protein_go_matrix.reset_index().merge(complex_map, on="Protein", how="inner")
go_by_complex = go_with_complex.drop(columns=["Protein"]).groupby("Complex_id").max()


Unir proteinas individuales y complejas en una sola matriz. Aqui se construye P.

In [44]:
# Extraer proteínas individuales
single_proteins = target_c[target_c["Target_type"] == "SINGLE PROTEIN"]["Node_id"]
go_single = protein_go_matrix.loc[protein_go_matrix.index.isin(single_proteins)].copy()
go_single.index.name = "Node_id"

# Renombrar índices para unir
go_by_complex.index.name = "Node_id"

# Unión final
X_GO = pd.concat([go_single, go_by_complex], axis=0)


In [45]:
# cuantas single_protein hay en X_GO
print(f"Total single proteins in X_GO: {go_single.shape[0]}")
# cuantos complejos hay en X_GO
print(f"Total complexes in X_GO: {go_by_complex.shape[0]}")

Total single proteins in X_GO: 3753
Total complexes in X_GO: 1408


In [46]:
# Matriz X.
print(X_GO.shape)  # (5161, 6928)

(5161, 6928)


In [47]:
# Guardamos 
output_dir = os.path.join("..","data","processed", "raw_cleaned")

dataset_completo = X_GO.reset_index()

# Guardar como CSV
dataset_completo.to_csv(os.path.join(output_dir, 'dataset_completo.csv'), index=False)

# Verificación
print("\nDatos guardados correctamente.")
print(f"Archivo guardado en: {os.path.join(output_dir, 'dataset_completo.csv')}")


Datos guardados correctamente.
Archivo guardado en: ..\data\processed\raw_cleaned\dataset_completo.csv


In [48]:
# Visualización dataset
print("\nPrimeras filas del dataset completo:")
print(dataset_completo.head())


Primeras filas del dataset completo:
  Node_id  GO:0000002  GO:0000012  GO:0000014  GO:0000018  GO:0000022  \
0   A2ML1           0           0           0           0           0   
1    AAK1           0           0           0           0           0   
2   AARS1           0           0           0           0           0   
3   AARS2           0           0           0           0           0   
4    AASS           0           0           0           0           0   

   GO:0000027  GO:0000028  GO:0000030  GO:0000038  ...  GO:2001244  \
0           0           0           0           0  ...           0   
1           0           0           0           0  ...           0   
2           0           0           0           0  ...           0   
3           0           0           0           0  ...           0   
4           0           0           0           0  ...           0   

   GO:2001251  GO:2001252  GO:2001256  GO:2001257  GO:2001258  GO:2001259  \
0           0           0

In [49]:
# Ver cuantos complejos y cuantas single_protein hay
print(f"Total single proteins in dataset: {dataset_completo[dataset_completo['Node_id'].isin(single_proteins)].shape[0]}")
print(f"Total complexes in dataset: {dataset_completo[dataset_completo['Node_id'].isin(complex_map['Complex_id'])].shape[0]}")

Total single proteins in dataset: 3753
Total complexes in dataset: 1408


## Construir GO.txt

In [50]:
output_dir = os.path.join("..", "data", "Input")

# Convertir X_GO a formato largo (Protein_ID, GO_term)
go_long = X_GO.reset_index().melt(id_vars="Node_id", var_name="GO_term", value_name="has_GO")

# Filtrar solo donde la proteína tiene ese término GO
go_long = go_long[go_long["has_GO"] == 1][["Node_id", "GO_term"]]

# Guardar
go_long.to_csv(os.path.join(output_dir, "GO.csv"), sep='\t', index=False, header=False)


In [51]:
num_go_terms_XGO = X_GO.shape[1]
print("Términos GO distintos en X_GO:", num_go_terms_XGO)


Términos GO distintos en X_GO: 6928


In [52]:
num_go_terms_GOtxt = go_long["GO_term"].nunique()
print("Términos GO en GO.txt:", num_go_terms_GOtxt)

Términos GO en GO.txt: 6928


In [53]:
# Todos los términos GO de X_GO (columnas)
go_terms_XGO = set(X_GO.columns)

# Todos los términos que aparecen en GO.txt
go_terms_GOtxt = set(go_long["GO_term"].unique())

# Diferencia: términos que están en X_GO pero no llegaron a GO.txt
missing_terms = go_terms_XGO - go_terms_GOtxt

print(f"Cantidad de términos GO perdidos: {len(missing_terms)}")
if missing_terms:
    print("Ejemplo de términos omitidos:", list(missing_terms)[:10])


Cantidad de términos GO perdidos: 0


In [54]:
# Contar cuántas veces aparece cada proteína
protein_counts = go_long["Node_id"].value_counts()

# Filtrar solo las proteínas que aparecen más de una vez
repeated_proteins = protein_counts[protein_counts > 1]

# Mostrar cuántas se repiten y un ejemplo
print("Número de proteínas que se repiten (tienen más de 1 GO):", repeated_proteins.shape[0])
print("\nProteínas más anotadas (top 10):")
print(repeated_proteins.head(10))


Número de proteínas que se repiten (tienen más de 1 GO): 5109

Proteínas más anotadas (top 10):
Node_id
CPX-623     338
CPX-5223    301
CPX-624     300
CPX-6015    279
CPX-467     278
CPX-978     262
CPX-439     258
CPX-3323    256
CPX-109     255
CPX-1080    241
Name: count, dtype: int64


In [55]:
# 1. Obtener proteínas que tienen más de un GO
protein_counts = go_long["Node_id"].value_counts()
repeated_proteins = protein_counts[protein_counts > 1].index.tolist()

# 2. Filtrar target_c por esas proteínas
repeated_info = target_c[target_c["Node_id"].isin(repeated_proteins)][["Node_id", "Target_type"]]

# 3. Contar cuántas son SINGLE y cuántas COMPLEX
repeated_summary = repeated_info["Target_type"].value_counts()

print("Clasificación de proteínas con más de un GO:")
print(repeated_summary)


Clasificación de proteínas con más de un GO:
Target_type
SINGLE PROTEIN     3701
PROTEIN COMPLEX    1408
Name: count, dtype: int64
